In [12]:
import pandas as pd
import json
import requests
from requests.auth import HTTPBasicAuth

from configparser import ConfigParser
parser = ConfigParser()

_ = parser.read("notebook.cfg")

login = parser.get("spnc", "login")
password = parser.get("spnc", "password")

In [13]:
resp = requests.get("https://service-public.nc/api/login-token", auth=HTTPBasicAuth(login, password))
token = resp.json()['access_token']
token

'Zb33_V0mj-50SOCoKh0zjBPuAKgVh2CBZDcPQtFPxhk'

In [14]:
data = []
flag = True
url = "https://service-public.nc/api/dossiers?access_token=%s" % token
while flag:
  res = requests.get(url)
  data.extend(res.json()['data'])
  links = res.json()['links']
  if "next" in links:
    url = res.json()['links']["next"]
  else:
    flag = False
  

print(len(data))
print(data[0])

396
{'type': 'dossiers', 'id': '163', 'attributes': {'nom': 'Permis de construire en province Sud', 'description': "Le permis de construire est un acte administratif qui donne les moyens à l'administration de vérifier qu'un projet de construction respecte bien les règles d'urbanisme en vigueur. En province Sud, tout projet de construction immobilière ou de rénovation d’un bâtiment est préalablement soumis, à l’autorisation de l’administration compétente, par le biais d’une demande de permis de construire. La démarche ci-dessous explique les formalités de demande de permis de construire dans les différentes communes de la province Sud.\nDéclaration préalable\xa0: Il est désormais possible de faire une simple déclaration préalable pour les ouvrages, constructions, aménagements, installations et travaux qui, en raison de leurs dimensions, de leur nature ou de leur localisation, ne justifient pas l’exigence d’un permis de construire.\n", 'url': 'https://service-public.nc/particuliers/logem

In [15]:
themes = {}
flag = True
url = "https://service-public.nc/api/themes?access_token=%s" % token
while flag:
  res = requests.get(url)
  for t in res.json()['data']:
    themes[t['id']] = t['attributes']['nom']
  links = res.json()['links']
  if "next" in links:
    url = res.json()['links']["next"]
  else:
    flag = False
  

print(len(themes))
themes

147


{'55': 'Particuliers',
 '56': 'Argent',
 '57': 'Impôts',
 '58': 'Consommation',
 '59': 'Travail',
 '60': "Recherche d'emploi",
 '61': 'Emploi',
 '63': 'Prestations familiales',
 '65': 'Papiers - Citoyenneté',
 '66': 'Etat civil',
 '67': 'Elections',
 '68': 'Santé - Social',
 '69': 'Couverture Santé',
 '71': 'Logement',
 '74': 'Transports',
 '75': 'Automobile',
 '78': 'Professionnels',
 '146': 'Fiscalité',
 '151': 'Dans la fonction publique',
 '156': 'Environnement',
 '157': 'ICPE',
 '159': 'Protection de la faune et de la flore',
 '160': "Demande d'autorisation et dérogation",
 '426': 'Permis de conduire',
 '429': 'Construction et Rénovation',
 '430': 'Allocation chômage',
 '432': 'Retraite',
 '434': 'Gestion - Finance',
 '435': "Gestion d'entreprise",
 '436': 'Marchés publics',
 '438': 'Ressources humaines',
 '439': 'Recrutement',
 '443': 'Sécurité sanitaire',
 '445': "Location d'un bien",
 '446': 'Dans le secteur privé',
 '447': "Gestion de l'eau",
 '449': 'Maritime',
 '450': 'Carte 

In [16]:
tmp = []
for t in themes:
    tmp.append([t, themes[t]])
themes = tmp

In [17]:
csv = []
# id, nom, description, url, 
for d in data:
    csv.append([d['id'], d['attributes']['nom'], d['attributes']['description'], d['attributes']['url'], list(map(lambda t: t['id'], d['relationships']['themes']['data']) ) ])
csv

[['163',
  'Permis de construire en province Sud',
  "Le permis de construire est un acte administratif qui donne les moyens à l'administration de vérifier qu'un projet de construction respecte bien les règles d'urbanisme en vigueur. En province Sud, tout projet de construction immobilière ou de rénovation d’un bâtiment est préalablement soumis, à l’autorisation de l’administration compétente, par le biais d’une demande de permis de construire. La démarche ci-dessous explique les formalités de demande de permis de construire dans les différentes communes de la province Sud.\nDéclaration préalable\xa0: Il est désormais possible de faire une simple déclaration préalable pour les ouvrages, constructions, aménagements, installations et travaux qui, en raison de leurs dimensions, de leur nature ou de leur localisation, ne justifient pas l’exigence d’un permis de construire.\n",
  'https://service-public.nc/particuliers/logement/construction-et-renovation/permis-de-construire-en-province-sud

In [18]:
from sklearn.preprocessing import MultiLabelBinarizer
df = pd.DataFrame(csv, columns=["id", "nom", "description", "url", "themes"])

mlb = MultiLabelBinarizer(sparse_output=True)

df = df.join(
            pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(df.pop('themes')),
                index=df.index,
                columns=mlb.classes_))
df.head()

,id,nom,description,url,146,151,156,157,159,160,...,63,65,66,67,68,69,71,74,75,78
0,163,Permis de construire en province Sud,Le permis de construire est un acte administra...,https://service-public.nc/particuliers/logemen...,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,170,Revalorisation des loyers,Les loyers ne sont pas des prestations de serv...,https://service-public.nc/particuliers/argent/...,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,175,Allocation chômage total,L'assurance chômage assure au salarié involont...,https://service-public.nc/particuliers/travail...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,176,Emploi de personnes à domicile,Les employés de maison (ou gens de maison) son...,https://service-public.nc/particuliers/travail...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,177,Pension de retraite des salariés du secteur privé,Une personne prend sa retraite lorsqu’elle arr...,https://service-public.nc/particuliers/travail...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
df = pd.DataFrame(csv, columns=["id", "nom", "description", "url", "themes"])

mlb = MultiLabelBinarizer()

df = df.join(
            pd.DataFrame(mlb.fit_transform(df.pop('themes')),
                          columns=mlb.classes_,
                          index=df.index))
df.head()

,id,nom,description,url,146,151,156,157,159,160,...,63,65,66,67,68,69,71,74,75,78
0,163,Permis de construire en province Sud,Le permis de construire est un acte administra...,https://service-public.nc/particuliers/logemen...,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,170,Revalorisation des loyers,Les loyers ne sont pas des prestations de serv...,https://service-public.nc/particuliers/argent/...,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,175,Allocation chômage total,L'assurance chômage assure au salarié involont...,https://service-public.nc/particuliers/travail...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,176,Emploi de personnes à domicile,Les employés de maison (ou gens de maison) son...,https://service-public.nc/particuliers/travail...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,177,Pension de retraite des salariés du secteur privé,Une personne prend sa retraite lorsqu’elle arr...,https://service-public.nc/particuliers/travail...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
for col in df.columns:
    if col in themes:
        df = df.rename(columns={col: themes[col]})
df.head()

,id,nom,description,url,146,151,156,157,159,160,...,63,65,66,67,68,69,71,74,75,78
0,163,Permis de construire en province Sud,Le permis de construire est un acte administra...,https://service-public.nc/particuliers/logemen...,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,170,Revalorisation des loyers,Les loyers ne sont pas des prestations de serv...,https://service-public.nc/particuliers/argent/...,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,175,Allocation chômage total,L'assurance chômage assure au salarié involont...,https://service-public.nc/particuliers/travail...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,176,Emploi de personnes à domicile,Les employés de maison (ou gens de maison) son...,https://service-public.nc/particuliers/travail...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,177,Pension de retraite des salariés du secteur privé,Une personne prend sa retraite lorsqu’elle arr...,https://service-public.nc/particuliers/travail...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
df.describe()

,146,151,156,157,159,160,426,429,430,432,...,63,65,66,67,68,69,71,74,75,78
count,396.000000,396.000000,396.000000,396.000000,396.000000,396.000000,396.000000,396.000000,396.000000,396.000000,...,396.000000,396.000000,396.000000,396.000000,396.000000,396.000000,396.000000,396.000000,396.000000,396.000000
mean,0.020202,0.012626,0.040404,0.002525,0.012626,0.015152,0.017677,0.030303,0.005051,0.015152,...,0.010101,0.098485,0.035354,0.005051,0.148990,0.060606,0.083333,0.068182,0.032828,0.292929
std,0.140869,0.111796,0.197154,0.050252,0.111796,0.122310,0.131940,0.171637,0.070977,0.122310,...,0.100121,0.298346,0.184905,0.070977,0.356529,0.238908,0.276735,0.252377,0.178412,0.455682
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [22]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size=0.2)

print(df_train.shape)
print(df_test.shape)

(316, 149)
(80, 149)


In [23]:
df_train.to_csv('data/train.csv')
df_test.to_csv('data/test.csv')